In [1]:
"""
PersianSwear Class
Author : Amir Shokri @amirshnll
created date : 11 October, 2021
updated date : 11 October, 2021
"""

'\nPersianSwear Class\nAuthor : Amir Shokri @amirshnll\ncreated date : 11 October, 2021\nupdated date : 11 October, 2021\n'

In [2]:
import json
import os.path

In [3]:
class PersianSwear(object):
	def __init__(self):
		self.data = json.load(open('data.json'))

	# return string
	def filter_words(self, text, symbol="*"):
		if(self.is_empty()):
			return text

		text = text.split()
		for i in range(len(text)):
			if text[i] in self.data['word']:
				text[i] = symbol

		return " ".join(text)

	# return boolean
	def is_empty(self):
		if(len(self.data['word'])<1):
			return True;
		return False;

	# return nothing
	def add_word(self, text):
		self.data['word'].append(text)

	# return nothing
	def remove_word(self, text):
		self.data['word'].remove(text)	

	# return boolean
	def is_bad(self, text):
		return text in self.data['word']

	# return boolean
	def has_swear(self, text):
		if(self.is_empty()):
			return text

		text = text.split()
		for i in range(len(text)):
			if text[i] in self.data['word']:
				return True

		return False

	# return string
	def tostring(self):
		return ' - '.join(self.data['word'])

In [4]:
persianswear = PersianSwear()

In [5]:
print(persianswear.is_bad('خر'))

True


In [6]:
print(persianswear.is_bad('امروز'))

False


In [7]:
print(persianswear.is_bad('چرت و پرت'))

False


In [8]:
persianswear.add_word('چرت و پرت')
print(persianswear.is_bad('چرت و پرت'))

True


In [9]:
print(persianswear.has_swear('تو دوست من هستی'))

False


In [10]:
print(persianswear.has_swear('تو هیز هستی'))

True


In [11]:
print(persianswear.filter_words('تو دوست من هستی'))

تو دوست من هستی


In [12]:
print(persianswear.filter_words('تو هیز هستی'))

تو * هستی


In [13]:
print(persianswear.filter_words('تو هیز هستی', '&'))

تو & هستی


In [14]:
print(persianswear.tostring())

آب کیر - آشغال - آلت تناسلی - آلت - ابله - ابن یزید - احمق - اسب - اسبی - اسکل - اسکل - اسگل - اسگول - الاغ - الاق - انگل - انی - انی - اوسکل - اوسکل - اوسگل - اوصکل - اوصگل - ب ک - باسن - بخورش - بدبخت - بمال - تخمم - کیرم - بپرروش - بپرسرش - کونی - بکارت - بکن توش - بکنش - بکنمت - خایه - بی عفت - بی غیرت - بی ناموس - بی پدر - بیابخورش - بیشعور - بیناموس - تخم سگ - تخمی - ترک - توله سگ - جاکش - جلق زدن - جنده - جنسی - جوون - جکس - جیندا - حرومزاده - حشر - حشری شدن - حشری - حیوانی - خارکس ده - خارکسده - خارکسّه - خانم جنده - خایه خور - خایه مال - خایه - خر - خرفت - خری - خز - خفه خون - خفه شو - خواهرجنده - خی کاس - داف ناز - داف - داگ استایل - دخترجنده - دخترقرتی - درازگوش - دله - دهن سرویس - گاییده - دهنت سرویس - دوجنسه - دول - دیوث - دیوس خان - دیوس - دیوص - رشتی - ریدن - ریدی - زارت - زباله - زرنزن - زن جنده - زن کاسده - زنا زاده - زنا - زنازاده - زنتو - زنشو - زنیکه - سادیسمی - ساک - ساکونی - سرخور - سرکیر - سسکی - سوراخ کون - سوراخ کون - سولاخ - سکس چت - سکس - سکسی باش - سکسی - سک